In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [46]:
### Load all the trained models
model = load_model('ann_model.h5')

## Load Scaler and Encoder
with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('ohe_geo.pkl', 'rb') as file:
    ohe_geo = pickle.load(file)

with open('scalar.pkl', 'rb') as file:
    scalar = pickle.load(file)

In [47]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [48]:
geo_encoded = ohe_geo.transform([[input_data['Geography']]])
geo_encoded_df = pd.DataFrame(geo_encoded, columns=ohe_geo.get_feature_names_out(['Geography']))
geo_encoded_df

/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [49]:
print("Classes in label encoder:", label_encoder_gender.classes_)

Classes in label encoder: ['Female' 'Male']


In [50]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [51]:
## Encode categorical variables
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [52]:
## Concatenation w ohe data 
input_df = pd.concat([input_df.drop('Geography', axis=1), geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [53]:
## Scaling the data now
ip_scaled = scalar.transform(input_df)
ip_scaled

array([[-0.52656699,  0.9056767 ,  0.10556909, -0.69439066, -0.26521916,
         0.79633021,  0.64706357,  0.96390112, -0.87010537,  0.99104014,
        -0.57273139, -0.57504086]])

In [54]:
## Prediction 
prediction = model.predict(ip_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step


In [55]:
prediction

array([[0.03695584]], dtype=float32)

In [57]:
probab = prediction[0][0]

In [58]:
if probab > 0.5:
    print("Customer is likely to churn:")
else:
    print("Customer is not likely to churn:")

Customer is not likely to churn:
